# Check Results of Any Given Model

Code: [github:lucasdavid/experiments/.../supervised/fine-tuning/best-artworks-of-all-time/results](https://github.com/lucasdavid/experiments/blob/main/notebooks/supervised/fine-tuning/best-artworks-of-all-time/results.ipynb)  
Dataset: https://www.kaggle.com/ikarus777/best-artworks-of-all-time  
Docker image: `tensorflow/tensorflow:latest-gpu-jupyter`  

In [ ]:
import os
import numpy as np

class RC:
    base_path = ('/tf/logs/baoat/d:baoat-patches e:200 fte:0 b:32 v:0.3 '
                 'm:inceptionv3 aug:True/1610567121')
    model = f'{base_path}/weights.h5'
    results = f'{base_path}/eval'
    
class DC:
    path = '/tf/datasets/best-artworks-of-all-time'
    images = path + '/images/images'
    classes = np.asarray(sorted(os.listdir(images)))

class Config:
    run = RC
    data = DC

## Setup

In [ ]:
import pathlib
from math import ceil

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def plot(y, titles=None, rows=1, i0=0):
    for i, image in enumerate(y):
        if image is None:
            plt.subplot(rows, ceil(len(y) / rows), i0+i+1)
            plt.axis('off')
            continue

        t = titles[i] if titles else None
        plt.subplot(rows, ceil(len(y) / rows), i0+i+1, title=t)
        plt.imshow(image)
        plt.axis('off')

In [ ]:
sns.set()

## Report

In [ ]:
results = pd.read_csv(Config.run.results + '/predictions.csv')
results.head()

In [ ]:
def count(x):
    p, c = np.unique(x.pred, return_counts=True)
    return p[np.argmax(c)]

predictions = pd.DataFrame(results.groupby('name').apply(count), columns=['pred'])
labels = results[['name', 'label']].drop_duplicates().set_index('name')
report = labels.join(predictions)
report['author'] = ['_'.join(n.split('_')[:-1]) for n in report.index.tolist()]

In [ ]:
report['correct'] = report.label == report.pred
report.head(2)

In [ ]:
report.correct.mean()

In [ ]:
from sklearn import metrics as skmetrics

l = Config.data.classes[report.label]
p = Config.data.classes[report.pred]

print(skmetrics.classification_report(l, p, labels=Config.data.classes))

In [ ]:
cm = skmetrics.confusion_matrix(l, p, labels=Config.data.classes)
cm = cm / cm.sum(axis=1, keepdims=True)

plt.figure(figsize=(12, 12))
with sns.axes_style("white"):
    sns.heatmap(cm, cmap='RdPu', annot=False, cbar=False, xticklabels=False,
                yticklabels=Config.data.classes);

plt.savefig(Config.run.results + '/test-cm.jpg')